## 加上每周数据
## 数据进行归一化
## 模型换成kaggle神器
## 要求：就是提升到前一百名

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

import gc
import time
import pickle
import seaborn as sns
from tqdm import tqdm
from itertools import product
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn import preprocessing 
from xgboost import plot_importance
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float16)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

import sys
sys.version_info

sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)

## 读入保存好的数据

In [2]:
data = pd.read_pickle('data2.pkl')
# data = data[[
#     'date_block_num', 
#     'shop_id', 
#     'item_id', 
#     'item_cnt_month',
    
# #     'shop_city', 
# #     'shop_name1',
# #     'shop_type', 
    
# #     'name_1', 
# #     'name_2', 
# #     'name_3',
    
# #     'item_type', 
# #     'item_subtype', 
# #     'item_category_id', 
    
# #     'item_cnt_month_lag_1',
# #     'item_cnt_month_lag_2', 
# #     'item_cnt_month_lag_3',
# #     'date_block_num_avg_item_cnt_lag_1',
# #     'date_block_num_and_item_id_avg_item_cnt_lag_1',
# #     'date_block_num_and_item_id_avg_item_cnt_lag_2',
# #     'date_block_num_and_item_id_avg_item_cnt_lag_3',
# #     'date_block_num_and_shop_id_avg_item_cnt_lag_1',
# #     'date_block_num_and_shop_id_avg_item_cnt_lag_2',
# #     'date_block_num_and_shop_id_avg_item_cnt_lag_3',
    
# #     'date_block_num_and_shop_city_avg_item_cnt_lag_1',
# #     'date_block_num_and_shop_name1_avg_item_cnt_lag_1',
# #     'date_block_num_and_shop_type_avg_item_cnt_lag_1',
    
# #     'date_block_num_and_item_category_id_avg_item_cnt_lag_1',
# #     'date_block_num_and_item_type_avg_item_cnt_lag_1',
# #     'date_block_num_and_item_subtype_avg_item_cnt_lag_1',
    
# #     'date_block_num_and_shop_id_and_item_id_avg_item_cnt_lag_1',
# #     'date_block_num_and_shop_id_and_name_1_avg_item_cnt_lag_1',
# #     'date_block_num_and_shop_id_and_name_2_avg_item_cnt_lag_1',
# #     'date_block_num_and_shop_id_and_name_3_avg_item_cnt_lag_1',
# #     'date_block_num_and_shop_id_and_item_category_id_avg_item_cnt_lag_1',
    
# #     'delta_price_lag', 
# #     'item_shop_last_sale', 
# #     'item_last_sale',
# #     'item_first_sale', 
# #     'year', 
# #     'month', 
# #     'days'
# ]]

print(data.shape)
data.head(3)

(7756739, 32)


,date_block_num,shop_id,item_id,item_cnt_month,shop_city,shop_type,item_category_id,name_1,name_2,name_3,item_type,item_subtype,year,month,days,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,date_block_num_avg_item_cnt_lag_1,date_block_num_and_item_id_avg_item_cnt_lag_1,date_block_num_and_item_id_avg_item_cnt_lag_2,date_block_num_and_item_id_avg_item_cnt_lag_3,date_block_num_and_shop_id_avg_item_cnt_lag_1,date_block_num_and_shop_id_avg_item_cnt_lag_2,date_block_num_and_shop_id_avg_item_cnt_lag_3,date_block_num_and_item_category_id_avg_item_cnt_lag_1,date_block_num_and_shop_id_and_item_category_id_avg_item_cnt_lag_1,date_block_num_and_shop_id_and_item_id_avg_item_cnt_lag_1,delta_price_lag,item_shop_last_sale,item_last_sale,item_first_sale
1103919,4,2,27,0.0,0,5,19,27,77,42,5,10,2013,4,31,0.0,0.0,0.0,0.292969,0.027771,0.083313,0.083313,0.076233,0.095459,0.063599,0.743652,0.500000,0.0,0.345459,1,1,4
1103920,4,2,28,0.0,0,5,30,28,108,42,8,55,2013,4,31,0.0,0.0,0.0,0.292969,0.111084,0.083313,0.194458,0.076233,0.095459,0.063599,1.081055,0.399902,0.0,0.259521,-1,1,4
1103921,4,2,29,0.0,0,5,23,29,124,42,5,16,2013,4,31,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-1,4,4


In [3]:
##求出每个月的均值销售额##shop_mean_months
a = []
for i in range(3,34):
    b = data[data.date_block_num==i]##取到每一个月
    c = b.sum()['item_cnt_month']#求和
    d = len(b.shop_id.unique())#长度（个数）
    a.append(c/d)#求得均值
print(a)

D:\Anaconda3\envs\tensorflow2.0\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


[nan, 2154.6486486486488, 2463.0, 2351.0810810810813, 2506.7297297297296, 2377.945945945946, 2258.7297297297296, 2416.7837837837837, 3432.815789473684, 2279.1052631578946, 2168.3684210526317, 2128.358974358974, 1720.1, 1770.275, 1814.35, 1673.4878048780488, 1881.8536585365853, 1645.6829268292684, 1702.439024390244, 1962.7560975609756, 2952.170731707317, 1940.4878048780488, 1661.2926829268292, 1621.9756097560976, 1397.8536585365853, 1351.9268292682927, 1354.3170731707316, 1371.439024390244, 1484.5853658536585, 1392.0731707317073, 1360.7380952380952]


In [4]:
len(a)

31

In [5]:
##添加周特征，周的每一天的特征
weekarr = []
t = 2
count = 0
for w in range(3):
    for i in [31,28,31,30,31,30,31,31,30,31,30,31]:
        a = [0,0,0,0,0,0,0,count]
        count+=1
        for j in range(i):
            a[t]+=1
            if t==6:
                t=-1
            t+=1
        weekarr.append(a)
weekarr = pd.DataFrame(np.vstack(weekarr), columns=['week0','week1','week2','week3','week4','week5','week6','date_block_num'])
data = pd.merge(data, weekarr, on=['date_block_num'], how='left')#加进去
del weekarr
gc.collect()

68

In [6]:
##数据集的划分
X_zong = data.drop(['item_cnt_month'], axis=1)#去掉标签
Y_train = data[data.date_block_num < 33]['item_cnt_month']#训练集的标签
Y_valid = data[data.date_block_num == 33]['item_cnt_month']#交叉验证的标签
del data##删除数据集减少占用内存
gc.collect()##垃圾回收机制

27

In [8]:
from sklearn.preprocessing import MinMaxScaler ##数据归一化
minMax = MinMaxScaler()    


X_zong_std = minMax.fit_transform(X_zong.iloc[:,:-7])  ##取所有的行，除了后七列的所有的
X_zong.iloc[:,:-7] = pd.DataFrame(np.vstack(X_zong_std),columns=X_zong.columns[:-7])
X_zong.iloc[:,:-7] = downcast_dtypes(X_zong.iloc[:,:-7])#转换数据类型，为了减少内存

##这就是排除出去的后七列
X_zong['week0'] = X_zong['week0'].astype(np.int8)
X_zong['week1'] = X_zong['week1'].astype(np.int8)
X_zong['week2'] = X_zong['week2'].astype(np.int8)
X_zong['week3'] = X_zong['week3'].astype(np.int8)
X_zong['week4'] = X_zong['week4'].astype(np.int8)
X_zong['week5'] = X_zong['week5'].astype(np.int8)
X_zong['week6'] = X_zong['week6'].astype(np.int8)

del X_zong_std
gc.collect()

MemoryError: 

In [ ]:
X_train = X_zong[X_zong.date_block_num < 0.96679688]
X_valid = X_zong[X_zong.date_block_num == 0.96679688]
X_test = X_zong[X_zong.date_block_num == 1]
del X_zong
gc.collect()

## 模型的训练--lightgbm、catboost ，集成（堆叠）

In [ ]:
##参数可以自己改
ts = time.time()

model = XGBRegressor(
    max_depth=10,
    n_estimators=1000,
    min_child_weight=0.5, 
    colsample_bytree=0.9, 
    subsample=0.8, 
    eta=0.1,    
    seed=1)

model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 10)

time.time() - ts

In [ ]:
plot_features(model, (10,14))

## 保存为自己所需要的数据

In [ ]:
Y_test = model.predict(X_test).clip(0, 20)##clip0-20之间
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv').set_index('ID')

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('xgb_submission1.csv', index=False)

## 自己先查资料、了解地震波的原理，以及以报告的形式，写出地震比赛中的要求以及评估方式，自己可以去探索一下数据（EDA）。